In [1]:
import torch
import numpy as np
import trimesh
import matplotlib.pyplot as plt
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [2]:
def load_point_cloud(file_path):
    """Carrega a nuvem de pontos usando Trimesh e retorna os vértices como tensor no dispositivo configurado."""
    mesh = trimesh.load(file_path)
    vertices = mesh.vertices
    return torch.tensor(vertices, dtype=torch.float32, device=device)

def cdist_batched(src, target, batch_size=512):
    results = []
    for i in range(0, src.shape[0], batch_size):
        src_batch = src[i:i+batch_size]
        d = torch.cdist(src_batch, target)
        results.append(d)
    return torch.cat(results, dim=0)


def icp(source, target, max_iter=50, tol=1e-5):
    """Implementa o algoritmo ICP para alinhar a nuvem de pontos 'source' à 'target'."""
    T_total = torch.eye(4, device=device)
    src = source.clone()
    
    for i in range(max_iter):
        # 1. Encontrar correspondências entre os pontos usando torch.cdist
        distances = cdist_batched(src, target)
        indices = torch.argmin(distances, dim=1)
        target_corr = target[indices]
        
        # 2. Calcular os centróides
        centroid_src = src.mean(dim=0)
        centroid_target = target_corr.mean(dim=0)
        
        # 3. Centralizar os pontos
        src_centered = src - centroid_src
        target_centered = target_corr - centroid_target
        
        # 4. Calcular a matriz de covariância
        H = src_centered.t() @ target_centered
        U, S, Vt = torch.linalg.svd(H)
        R = Vt.t() @ U.t()
        
        # Corrigir reflexão, se necessário
        if torch.det(R) < 0:
            Vt[-1, :] *= -1
            R = Vt.t() @ U.t()
        
        # 5. Calcular a translação
        t = centroid_target - R @ centroid_src
        
        # 6. Formar a matriz de transformação homogênea
        T = torch.eye(4, device=device)
        T[:3, :3] = R
        T[:3, 3] = t
        
        # 7. Aplicar a transformação na nuvem de pontos source
        ones = torch.ones(src.shape[0], 1, device=device)
        src_h = torch.cat([src, ones], dim=1)  # converter para coordenadas homogêneas
        src_h = (T @ src_h.t()).t()
        src = src_h[:, :3]
        
        # Acumular a transformação
        T_total = T @ T_total
        
        # Critério de convergência: se a média dos valores singulares for menor que tol
        if S.mean() < tol:
            break
    
    return T_total


In [3]:
# Defina o diretório base dos arquivos
point_cloud_dir = "objetos/KITTI-Sequence"  # Diretório base onde estão as pastas dos scans
ground_truth_file = "./ground_truth.npy"  # Mantenha ou ajuste se necessário

num_scans = 10
trajectory = [torch.eye(4, device=device)]

for i in range(1, num_scans):
    source_file = os.path.join(point_cloud_dir, f"{i-1:06d}", f"{i-1:06d}_points.obj")
    target_file = os.path.join(point_cloud_dir, f"{i:06d}", f"{i:06d}_points.obj")
    
    if not os.path.exists(source_file) or not os.path.exists(target_file):
        print(f"Arquivo não encontrado: {source_file} ou {target_file}")
        continue

    source = load_point_cloud(source_file)
    target = load_point_cloud(target_file)

    T = icp(source, target)
    T_acc = T @ trajectory[-1]
    trajectory.append(T_acc)

# Libera as variáveis usadas neste loop
del source, target, T
torch.cuda.empty_cache()
import gc; gc.collect()


# Converter as matrizes de transformação para CPU para visualização
trajectory_cpu = [T.cpu().numpy() for T in trajectory]
positions = np.array([T[:3, 3] for T in trajectory_cpu])

# Plotar a trajetória estimada em 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], marker='o')
ax.set_title("Trajetória Estimada do Veículo")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.show()

# Carregar a ground truth e comparar
if os.path.exists(ground_truth_file):
    ground_truth = np.load(ground_truth_file)  # Espera-se um array de tamanho (30, 4, 4)
    errors = []
    for T_est, T_gt in zip(trajectory_cpu, ground_truth):
        pos_est = T_est[:3, 3]
        pos_gt = T_gt[:3, 3]
        error = np.linalg.norm(pos_est - pos_gt)
        errors.append(error)
    print("Erro médio de translação:", np.mean(errors))
else:
    print("Arquivo de ground truth não encontrado.")

OutOfMemoryError: CUDA out of memory. Tried to allocate 14.53 GiB. GPU 0 has a total capacity of 23.56 GiB of which 8.66 GiB is free. Including non-PyTorch memory, this process has 14.89 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 13.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


NameError: name 'reorient_points' is not defined